In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from mlresearch.utils import set_matplotlib_style
from recgame.environments import BaseEnvironment
from recgame.recourse import NFeatureRecourse
from recgame.utils import generate_synthetic_data

In [2]:
set_matplotlib_style(16)
# SAVE_PATH = "../assets/multi-agent-dynamic-counterfactual-recourse/analysis/"

# Some variables to define the problem
RNG_SEED = 42
rng = np.random.default_rng(RNG_SEED)
N_CONTINUOUS = 2
N_CAT = 0
N_AGENTS = 1000

# Environment variables
N_LOANS = 10  # np.array([20, 1, 10, 30, 0, 5, 50, 10, 40])
ADAPTATION = 0.1  # 0.5  # np.array([.2, .1, .3, .25, .1, .4, .05, .05, .2])
NEW_AGENTS = 10  # np.array([40, 10, 50, 5, 0, 100, 0, 0, 20])

df, y, categorical = generate_synthetic_data(N_AGENTS, N_CONTINUOUS, N_CAT, RNG_SEED)

lr = LogisticRegression().fit(df, y)

In [3]:
def data_source_func(n_agents):
    X, _, _ = generate_synthetic_data(
        n_agents=n_agents,
        n_continuous=N_CONTINUOUS,
        n_categorical=N_CAT,
        random_state=rng
    )
    return X

In [4]:
# Define the necessary components to run simulation
recourse = NFeatureRecourse(lr, n_features=2, categorical=categorical)
environment = BaseEnvironment(
    X=df,
    recourse=recourse,
    data_source_func=data_source_func,
    threshold=N_LOANS,
    threshold_type="absolute",
    adaptation=ADAPTATION,
    behavior_function="continuous_constant",
    growth_rate=NEW_AGENTS,
    growth_rate_type="absolute",
    random_state=RNG_SEED,
)

environment.simulate(10)

IndexError: positional indexers are out-of-bounds

In [ ]:
environment.plot.agent_scores()

In [ ]:
set_matplotlib_style(16, **{"lines.markersize": 5})
fig, ax = plt.subplots(1, 1, figsize=[6.4, 6.4])
ax = environment.plot.scatter(5, title=False, ax=ax)
ax.set_xlabel("Feature 0")
ax.set_ylabel("Feature 1")
plt.savefig(
    join(SAVE_PATH, f"agents_scatter_plot_adaptation_{ADAPTATION}_new_agents_{NEW_AGENTS}.pdf"),
    format="pdf",
    bbox_inches="tight",
)
plt.show()

In [ ]:
environment.plot.scores_histogram(0)

In [ ]:
set_matplotlib_style(16)
ax = environment.plot.scores_kde(0, color="#9abff1", legend=True, label="0")
ax = environment.plot.scores_kde(25, color="#2167bc", legend=True, label="25", ax=ax)
ax = environment.plot.scores_kde(50, color="#1f2c3d", legend=True, label="50", ax=ax)
ax.set_xlabel("Score")
ax.legend(title="Time Step")
plt.savefig(
    join(SAVE_PATH, f"agents_scores_distribution_adaptation_{ADAPTATION}_new_agents_{NEW_AGENTS}.pdf"),
    format="pdf",
    bbox_inches="tight",
)
plt.show()

In [ ]:
environment.plot.population_size()

In [ ]:
environment.success_rate(1, 20)

In [ ]:
environment.threshold_drift(1, 20)

In [ ]:
steps_info = environment.steps_info()
steps_info.tail(10)

In [ ]:
predictions = pd.concat(
    [
        steps_info.success_proba.iloc[:-1].reset_index(drop=True),
        steps_info.threshold_drift.iloc[1:].reset_index(drop=True) * -1,
    ],
    axis=1,
).astype(float)
((predictions - predictions.min()) / (predictions.max() - predictions.min())).iloc[
    :50
].plot.line()
predictions.corr()

In [ ]:
steps_info.astype(float).corr().round(2)

In [ ]:
agents_info = environment.agents_info()
agents_info

In [ ]:
agents_info.corr()

In [ ]:
from collections import Counter

# agents_info.plot.bar("n_failures", "n_adaptations")
agents_info_ = agents_info.copy()

agents_info_ = agents_info_[~agents_info_.favorable_step.isna()|agents_info_.n_failures > 0]
agents_info_["Initial Score"] = pd.cut(agents_info_["original_score"], 5, labels=["Lowest", "Low", "Mean", "High", "Highest"])
# agents_info_["success_rate"] = 

agents_info_.groupby(["Initial Score"]).mean()[["n_adaptations", "final_score", "n_failures"]]

ax = agents_info_.groupby(["Initial Score"]).mean()["n_failures"].plot.bar()
ax.set_ylabel("\# Failures (mean)")